In [5]:

import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/03/01 14:18:02 WARN Utils: Your hostname, MANADT083L resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/01 14:18:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 14:18:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -O ../data/yellow_tripdata_2024-10.parquet

In [15]:
df = spark.read \
    .option('header', 'true') \
    .parquet('../data/yellow_tripdata_2024-10.parquet')

In [17]:
for t in df.schema:
    print(f'field {t.name}\ttype {t.dataType}')

field VendorID	type IntegerType()
field tpep_pickup_datetime	type TimestampNTZType()
field tpep_dropoff_datetime	type TimestampNTZType()
field passenger_count	type LongType()
field trip_distance	type DoubleType()
field RatecodeID	type LongType()
field store_and_fwd_flag	type StringType()
field PULocationID	type IntegerType()
field DOLocationID	type IntegerType()
field payment_type	type LongType()
field fare_amount	type DoubleType()
field extra	type DoubleType()
field mta_tax	type DoubleType()
field tip_amount	type DoubleType()
field tolls_amount	type DoubleType()
field improvement_surcharge	type DoubleType()
field total_amount	type DoubleType()
field congestion_surcharge	type DoubleType()
field Airport_fee	type DoubleType()


In [12]:
schema = types.StructType([
    types.StructField('VendorID', types.ShortType(), True),
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True),
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True),
    types.StructField('passenger_count', types.LongType(), True),
    types.StructField('trip_distance', types.FloatType(), True),
    types.StructField('RatecodeID', types.ShortType(), True),
    types.StructField('store_and_fwd_flag', types.StringType(), True),
    types.StructField('PULocationID', types.ShortType(), True),
    types.StructField('DOLocationID', types.ShortType(), True),
    types.StructField('payment_type', types.ByteType(), True),
    types.StructField('fare_amount', types.FloatType(), True),
    types.StructField('extra', types.FloatType(), True),
    types.StructField('mta_tax', types.FloatType(), True),
    types.StructField('tip_amount', types.FloatType(), True),
    types.StructField('tolls_amount', types.FloatType(), True),
    types.StructField('improvement_surcharge', types.FloatType(), True),
    types.StructField('total_amount', types.FloatType(), True),
    types.StructField('congestion_surcharge', types.FloatType(), True),
    types.StructField('Airport_fee', types.FloatType(), True),
])

In [13]:
df = spark.read \
    .option('header', 'true') \
    .parquet('../data/yellow_tripdata_2024-10.parquet') 
for t in df.schema:
    print(t)

StructField('VendorID', ShortType(), True)
StructField('tpep_pickup_datetime', TimestampType(), True)
StructField('tpep_dropoff_datetime', TimestampType(), True)
StructField('passenger_count', LongType(), True)
StructField('trip_distance', FloatType(), True)
StructField('RatecodeID', ShortType(), True)
StructField('store_and_fwd_flag', StringType(), True)
StructField('PULocationID', ShortType(), True)
StructField('DOLocationID', ShortType(), True)
StructField('payment_type', ByteType(), True)
StructField('fare_amount', FloatType(), True)
StructField('extra', FloatType(), True)
StructField('mta_tax', FloatType(), True)
StructField('tip_amount', FloatType(), True)
StructField('tolls_amount', FloatType(), True)
StructField('improvement_surcharge', FloatType(), True)
StructField('total_amount', FloatType(), True)
StructField('congestion_surcharge', FloatType(), True)
StructField('Airport_fee', FloatType(), True)


Spark cannot alter the schema of the parquet on read. It must be converted post read

In [18]:
for field in schema:
    df = df.withColumn(field.name, df[field.name].cast(field.dataType))
for t in df.schema:
    print(f'field {t.name}\tnew dtype {t.dataType}')


field VendorID	new dtype ShortType()
field tpep_pickup_datetime	new dtype TimestampType()
field tpep_dropoff_datetime	new dtype TimestampType()
field passenger_count	new dtype LongType()
field trip_distance	new dtype FloatType()
field RatecodeID	new dtype ShortType()
field store_and_fwd_flag	new dtype StringType()
field PULocationID	new dtype ShortType()
field DOLocationID	new dtype ShortType()
field payment_type	new dtype ByteType()
field fare_amount	new dtype FloatType()
field extra	new dtype FloatType()
field mta_tax	new dtype FloatType()
field tip_amount	new dtype FloatType()
field tolls_amount	new dtype FloatType()
field improvement_surcharge	new dtype FloatType()
field total_amount	new dtype FloatType()
field congestion_surcharge	new dtype FloatType()
field Airport_fee	new dtype FloatType()


In [ ]:
df_part = df.repartition(4)

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/klang/de-zoom-2025/data/yellow already exists. Set mode as "overwrite" to overwrite the existing path.

In [21]:
df_part.write.parquet('../data/yellow')


## 2 partitioning

25 MB x 4 partitions

## 3 - count

- `pyspark.sql.functions.to_date` to convert timestamp to date
- `df.filter()` method

In [31]:
from pyspark.sql.functions import to_date
df.filter(to_date(df.tpep_pickup_datetime) == '2024-10-15').count()

128893

In [33]:
from pyspark.sql.functions import max, expr

In [34]:
df = df.withColumn(
    'trip_duration_sec', 
    expr('timestampdiff(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime)'))
df.select(max('trip_duration_sec')).show()


+----------------------+
|max(trip_duration_sec)|
+----------------------+
|                585424|
+----------------------+



In [36]:
585424 / 3600

162.61777777777777

## 4 longest trip duration

162 hours

## 5 UI

default port is 4040

In [37]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -O ../data/taxi_zone_lookup.csv

--2025-03-01 15:14:06--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.160, 3.164.82.40, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘../data/taxi_zone_lookup.csv’

../data/taxi_zone_l 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-01 15:14:07 (148 MB/s) - ‘../data/taxi_zone_lookup.csv’ saved [12331/12331]



In [38]:
zones = spark.read \
    .option('header', 'true') \
    .csv('../data/taxi_zone_lookup.csv')

zones.createOrReplaceTempView('zones')
res = spark.sql("select * from zones limit 5")
res.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [42]:
df.createOrReplaceTempView('trips')
res = spark.sql(
    "select PULocationID, count(*) numtrips, z.Zone "
    "from trips t "
    "join zones z "
    "on t.PULocationID = z.LocationID "
    "group by t.PULocationID, z.Zone "
    "order by numtrips"
)
res.show()

+------------+--------+--------------------+
|PULocationID|numtrips|                Zone|
+------------+--------+--------------------+
|         105|       1|Governor's Island...|
|         199|       2|       Rikers Island|
|           5|       2|       Arden Heights|
|           2|       3|         Jamaica Bay|
|         111|       3| Green-Wood Cemetery|
|         245|       4|       West Brighton|
|         204|       4|   Rossville/Woodrow|
|          44|       4|Charleston/Totten...|
|          84|       4|Eltingville/Annad...|
|         187|       4|       Port Richmond|
|          59|       6|        Crotona Park|
|         109|       6|         Great Kills|
|         156|       7|     Mariners Harbor|
|         118|       7|Heartland Village...|
|         206|       9|Saint George/New ...|
|         176|       9|             Oakwood|
|         172|      10|New Dorp/Midland ...|
|          30|      10|       Broad Channel|
|         184|      12|     Pelham Bay Park|
|         

## 6 - least freq pickup zone

- create another view from df
- use regular SQL to join zone lookup
- governor's island